# Step 1

In [1]:
import sys
import os
from os.path import join
import time
from datetime import datetime
import importlib
import numpy as np
import pandas as pd
import h5py
import imageio
from scipy import ndimage
from scipy import interpolate
import skimage
from tqdm import tqdm
from tqdm import trange
from matplotlib import pyplot as plt
from matplotlib import cm as cm
from matplotlib import colors
from matplotlib import patches
import proplot as pplt

# Local
sys.path.append('..')
from tools import energyVS06 as energy
from tools.plotting import plot_profiles
from tools.plotting import plot_image
from tools.plotting import plot_compare_images
from tools import image_processing as ip
from tools.utils import get_bins

In [ ]:
pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'

## Setup 

In [ ]:
cam = 'cam34'
datadir = '/Diagnostics/Data/Measurements/scan-xxpy-image-ypdE/2022-04-29/'
filenames = os.listdir(datadir)
filenames

In [ ]:
filename = '220429190854-scan-xxpy-image-ypdE'
file = h5py.File(join(datadir, 'preproc-' + filename + '.h5'), 'r')
data_sc = file['/scalar_data']
data_wf = file['/waveform_data']
data_im = file['/image_data']

print('All attributes:')
print()
for data in [data_sc, data_wf, data_im]:
    print(data.name)
    for item in data.dtype.fields.items():
        print(item)
    print()

In [ ]:
acts = data_sc.dtype.names[:3]
print('Actuators:')
print(acts)

print('Scalar diagnostics:')
sdiag = data_sc.dtype.names[3:6]
print(sdiag)

In [ ]:
signal = np.copy(data_sc[:, cam + '_Integral'])
ipeak = np.argmax(signal)
print('Index of peak signal:', ipeak)

### Mask BCM current

In [ ]:
# Gather the BCM data.
pv = 'bcm04'
bcm_data = data_sc[pv]
idx = np.arange(len(data))
times = [datetime.fromtimestamp(data_sc[i, 'timestamp']) for i in idx]
times = np.array(times)
print(f'Average BCM current (before masking) = {np.mean(bcm_data):.3f} +- {np.std(bcm_data):.3f} [mA]')

# Mask any low-current points.
bcm_limit = 25.0  # [mA]
idx_mask, = np.where(bcm_data > -bcm_limit)
idx_valid, = np.where(~np.isin(idx, idx_mask))
print('idx_mask:', idx_mask)
print(f'{len(idx_mask)} points masked due to BCM current < {bcm_limit:.3f} [mA]')
print(f'Average BCM current (after masking) = {np.mean(bcm_data[idx_valid]):.3f} +- {np.std(bcm_data[idx_valid]):.3f} [mA]')

# After masking, reselect the index of the peak signal.
ipeak = idx_valid[np.argmax(signal[idx_valid])]
print('New index of peak signal =', ipeak)

# Plot the BCM current during the scan, along with masked points.
fig, ax = pplt.subplots(figsize=(8.0, 2.0))
ax.plot(bcm_data[idx], color='black')
ax.plot(idx_mask, bcm_data[idx_mask], color='red', lw=0, marker='.', label='Masked')
ax.format(xlabel='Point', ylabel='BCM current [mA]', ygrid=True)
ax.legend(loc='upper left')
plt.show()

### Fit correlations

In [ ]:
corr = np.identity(len(acts))
intercept = np.zeros(corr.shape)
for i, act1 in enumerate(acts):
    for j, act2 in enumerate(acts):
        fit1d = np.polyfit(data_sc[:, act1], data_sc[:, act2], 1, w=signal)
        print(f'{act2} vs. {act1}: slope = {fit1d[0]:.3f}')
        corr[i, j] = fit1d[0]
        intercept[i, j] = fit1d[1]
print('Correlation matrix:')
print(corr)

In [ ]:
# fig, axes = pplt.subplots(nrows=3, ncols=3, figwidth=4.0, spanx=False, spany=False)
# for i in range(3):
#     for j in range(3):
#         ax = axes[i, j]
#         x = data_sc[acts[j]]
#         y = data_sc[acts[i]]
#         ax.plot(x, y, color='black', alpha=0.3)
#         xmin, xmax = np.min(x), np.max(x)
#         _x = x - 0.5 * (xmax - xmin)
#         _y = corr[j, i] * _x + intercept[j, i]
#         ax.plot(x, _y, color='black')
# for i, act in enumerate(acts):
#     axes[-1, i].set_xlabel(act)
#     axes[i, 0].set_ylabel(act)
# plt.show()

### Image processing

In [ ]:
ny = 512
nx = 612
y3 = np.arange(ny)
x3 = np.arange(nx)

def get_image(index):
    return data[index, cam + '_Image'].reshape(ny, nx)

In [ ]:
im = get_image(ipeak)

In [ ]:
edges = dict(l=140, r=100, b=130, t=80)
im1 = ip.crop(im, **edges)

axes = plot_compare_images(im, im1)
for ax in axes[:, 0]:
    width = im.shape[1] - (edges['l'] + edges['r'])
    height = im.shape[0] - (edges['t'] + edges['b'])
    center = (edges['l'], edges['b'])
    ax.add_patch(patches.Rectangle(center, width, height, fill=False, ec='red'))
plt.show()

In [ ]:
downscale_factor = 4
im = np.copy(im1)
im1 = skimage.transform.downscale_local_mean(
    im, 
    (downscale_factor, downscale_factor)
)
print(f'New image shape =', im1.shape)
print(f'im.nbytes / im_d.nbytes = {(im.nbytes / im1.nbytes):.3f}')

axes = plot_compare_images(im, im1)
axes.format(toplabels=['Original', f'Downscaled by factor {downscale_factor}'])

In [ ]:
thresh = 1000
thresh_frac_peak = thresh / np.max(im)
im = np.copy(im1)
im1 = ip.thresh(im, thresh, mask=True)

axes = plot_compare_images(im, im1)

In [ ]:
irow, icol = np.where(im == np.max(im))
i, j = irow[0], icol[0]
fig, axes = pplt.subplots(ncols=3, sharey=False)
axes[0].pcolormesh(im)
kws = dict(color='red', alpha=0.4)
axes[0].axhline(i, **kws)
axes[0].axvline(j, **kws)
axes[1].set_title(f'Row {i}')
axes[2].set_title(f'Column {j}')
kws = dict(color='black', lw=None)
axes[1].plot(np.arange(im.shape[1]), im[i, :] / np.max(im[i, :]), **kws)
axes[2].plot(np.arange(im.shape[0]), im[:, j] / np.max(im[:, j]), **kws)
axes[1].axhline(thresh / np.max(im[i, :]), color='black', alpha=0.1)
axes[2].axhline(thresh / np.max(im[:, j]), color='black', alpha=0.1)
axes[1:].format(yscale='log')
plt.show()

Now we will process all the images.

In [ ]:
images = []
for index in trange(len(data)):
    im = get_image(index)
    im = ip.crop(im, **edges)
    im = skimage.transform.downscale_local_mean(
        im, 
        (downscale_factor, downscale_factor),
    )
    im[im < thresh] = 0
    images.append(im)
images = np.array(images)

In [ ]:
print(f'Processed image array size = {images.nbytes:.2e} bytes')

## Interpolation

Work in the "normalized" (decorrelated) slit-screen coordinates. First, try transforming back to square grid.

In [ ]:
# Shearing matrix 
ndim = 3
M = np.identity(ndim)
M[1, 2] = 1.9  # dxp / dx
Minv = np.linalg.inv(M)

# y
d1_center = 6.5
d1_distance = 14.5
d1_steps = 32

# xp
d2_center = 15.0
d2_distance = 24.0
d2_steps = 32

# x
d3_center = 21.0
d3_distance = 17.5
d3_steps = 32

center = np.array([d1_center, d2_center, d3_center])
center = center[:, np.newaxis]
center

Test on csv file (planned points).

In [ ]:
names = []
for i in range(1, ndim + 1):
    names.extend([f'start{i}', f'stop{i}', f'step{i}'])
df = pd.read_csv(join(datadir, filename + '.csv'), 
                 skiprows=25, sep='\s+', header=None, names=names)
df

In [ ]:
# Extract actuator points.
act_pts = [df.loc[:, [f'start{i}', f'stop{i}']].values.ravel() 
           for i in range(1, ndim + 1)]
act_pts = np.array(act_pts).T

# Undo linear transformation.
act_pts_n = np.apply_along_axis(lambda pt: np.matmul(Minv, pt), 1, act_pts)

# Plot actuator points.
fig, axes = pplt.subplots(nrows=3, figsize=(8.0, 4.0), spany=False, aligny=True)
for i, ax in enumerate(axes):
    ax.plot(act_pts_n[:, i], color='black', lw=None, marker='.', ms=0)
    ax.format(ylabel=f'actuator {i}')
# plt.savefig('_output/actuators')

Create dataframe of actuator points.

In [ ]:
dim_names = ['y1', 'x2', 'x1']
columns = []
for i, dim_name in enumerate(dim_names):
    columns.extend([dim_name, dim_name + '_n'])
df2 = pd.DataFrame(index=df.index, columns=columns)
for i, dim_name in enumerate(dim_names):
    df2[dim_name] = act_pts[::2, i]
    df2[dim_name + '_n'] = act_pts_n[::2, i]
df2

### Procedure for one bin

#### Group by x,x' for a specified bin index

In [ ]:
nstep = 21
GV = []  # Bin positions
BI = []  # Bin indices
for i, dim_name in enumerate(dim_names):
    gv, idx = get_bins(act_pts_n, i, nstep, n_bins_mult=15)
    GV.append(gv)
    BI.append(idx)
    print(f'{dim_name}: {len(gv)} bins')

In [ ]:
# fig, axes = pplt.subplots(nrows=3, figsize=(8, 7), spany=False)
# for i, (ax, dim_name) in enumerate(zip(axes, dim_names)):
#     ax.plot(act_pts_n[:, i], marker='.', color='black', ms=3, lw=0)
#     ax.format(ylabel=dim_name, yticks=GV[i], ygrid=True, ytickminor=False, yticklabels=[])
# plt.show()

In [ ]:
GV2, GV1 = np.meshgrid(GV[2], GV[1], indexing='ij')

fig, ax = pplt.subplots(figwidth=4.5)
kws = dict(color='black', alpha=0.15)
for gv in GV[1]:
    ax.axvline(gv, **kws)
for gv in GV[2]:
    ax.axhline(gv, **kws)
ax.scatter(act_pts_n[:, 1], act_pts_n[:, 2], c='black', s=4, ec='None'); 
plt.show()

#### Group by x,x' coordinates

In [ ]:
# POINTS2D holds iteration number on x1-x2 grid (total iterations = 1/2 total sweeps)
# ordering: y, xp, x, yp, I
POINTS2D = np.full((len(GV[2]), len(GV[1])), np.nan)
counter = 1
steps = [0]
for i in range(POINTS2D.shape[0]):
    for j in range(POINTS2D.shape[1]):
        idx, = np.where((BI[1] == j) & (BI[2] == i))
        if len(idx) > 0:
            steps.append(idx)
            POINTS2D[i, j] = counter
        else:
            print(f'Bin {i},{j} empty')
        counter += 1
POINTS2D = POINTS2D.astype(int)

In [ ]:
idx_bin = []
for i, step in enumerate(steps):
    this_iter = np.unique(step / 2 + 1)
    if type(this_iter) is float:
        this_iter = np.array([this_iter])
    this_iter = this_iter.astype(int)
    idx_i = []
    for j in this_iter:
        this_idx, = np.where(data_sc['iteration'] == j)
        idx_i.append(this_idx)
    idx_bin.append(np.unique(np.hstack(idx_i)))

### Look at a single bin

In [ ]:
iteration_peak, n_iterations = None, len(steps)
for iteration in range(n_iterations):
    if ipeak in idx_bin[iteration]:
        iteration_peak = iteration
        print('peak in iteration %i'%iteration)

In [ ]:
iteration = iteration_peak

bcm_mean = np.mean(data_sc[:, 'bcm04'])
scale = bcm_mean / data_sc[idx_bin[iteration], 'bcm04']

fig, ax = pplt.subplots(figsize=(5.0, 2))
ax.plot(
    data_sc[idx_bin[iteration], 'y_PositionSync'],
    data_sc[idx_bin[iteration], cam + '_Integral'],
    color='black', marker='.',
)
ax.plot(
    data_sc[idx_bin[iteration], 'y_PositionSync'],
    data_sc[idx_bin[iteration], cam + '_Integral'] * scale,
    color='black', marker='.', alpha=0.2,
    label='Scaled by BCM04',
)
ax.legend(loc='upper right')
ax.format(xlabel='y_PositionSync', ylabel=cam+'_Integral')
plt.show()

### Make a gif of sweep

In [ ]:
i_peak_sweep = np.argmax(data_sc[idx_bin[iteration], cam + '_Integral'])
norm_pixel_value = np.max(data_im[idx_bin[iteration][i_peak_sweep], cam + '_Image'])
cmap = pplt.Colormap('viridis')
greyscale = False
log = False

_ims = []
for i in tqdm(idx_bin[iteration]):
    _im = images[i] / norm_pixel_value
    if log:
        _im = np.log10(_im + 1e-6)
    if not greyscale:
        _im = cmap(_im)
    _ims.append(np.uint8(_im * np.iinfo(np.uint8).max))
imageio.mimsave(f'_output/iteration{iteration}.gif', _ims, fps=6)

### Construct background image

This is done by collecting frames around the edge of the image. (This is not used anywhere in the analysis.)

In [ ]:
n_bins = 32
ny, nx = images[0].shape
edge_bins = np.arange(n_bins + 1, (n_bins + 1) * (n_bins + 1), n_bins + 1)
im_edge = np.zeros((ny, nx))
im_bg = np.zeros((edge_bins.shape[0], ny, nx))
for i, edge_bin in enumerate(edge_bins):
    nframes = len(idx_bin[edge_bin])
    im3d = np.zeros((nframes, ny, nx))
    for j in range(nframes):
        im3d[j, :, :] = images[idx_bin[edge_bin][j]]
    im_bg[i, :, :] = np.mean(im3d, axis=0)
im_bg = np.mean(im_bg, axis=0)
im_bg = ndimage.uniform_filter(im_bg, size=(4, 4), mode='nearest')

fig, ax = pplt.subplots()
ax.pcolormesh(im_bg)
ax.format(title='Background')
plt.show()

### Smooth and interpolate along y_PositionSync 

We have snapped all points onto an x1-x2 grid. What remains is to interpolate the y1 coordinate — the sweeping variable. We will do this by applying a filter to the 3D image.

In [ ]:
# Make the 3D image. 
n_frames = len(idx_bin[iteration])
im3d = np.zeros((n_frames, ny, nx))
for i in trange(n_frames):
    im3d[i, :, :] = images[idx_bin[iteration][i]]  
    
# Get y slit position at each frame.
yvals = data_sc[:, 'y_PositionSync'][idx_bin[iteration]]

# Apply a smoothing filter along the first dimension (y_PositionSync). We can do the same for 
# the camera dimensions; the time cost is the same.
im3d_smooth = ndimage.median_filter(im3d, size=(3, 1, 1), mode='constant', cval=0.0) 

In [ ]:
i = 25

fig, axes = pplt.subplots(ncols=2)
axes[0].pcolormesh(im3d[i, :, :])
axes[1].pcolormesh(im3d_smooth[i, :, :])
axes.format(toplabels=['Raw', 'Smoothed along y'])
plt.show()

#### Interpolate to y grid

In [ ]:
npts_y = 32
ygv = np.linspace(y_vals.min(),y_vals.max(),npts_y)

flip = False
if y_vals[0] > y_vals[-1]:
    flip = True
    y_vals = np.flip(y_vals)
    im3d_smooth = np.flip(im3d_smooth, axis=0)
    im3d = np.flip(im3d, axis=0)

# -- deal with any repeating points (otherwise interpolate won't work)
repeat_pts, = np.where(np.diff(y_vals) == 0)
counter = 1
for repeat_pt in reversed(repeat_pts):
    yvals[repeat_pt] -= 1e-7 * counter
    counter += 1

yf, ypf, xf = np.meshgrid(ygv, ypix, xpix, indexing='ij')
newpoints = np.vstack([yf.flatten(),ypf.flatten(),xf.flatten()]).T
arr3d = interpolate.interpn((y_vals,ypix,xpix), im3d_smooth, newpoints, method='linear', bounds_error=False, fill_value=0.)
arr3d = np.reshape(arr3d,[npts_y,ny,nx])

In [ ]:
plt.figure()
plt.plot(y_vals,im3d.sum(axis=1).sum(axis=1),'.',label='each point')
plt.plot(y_vals,im3d_smooth.sum(axis=1).sum(axis=1),'-',linewidth=2,label='smoothed')
plt.plot(ygv,arr3d.sum(axis=1).sum(axis=1),'-k',label='interpolated')
plt.legend()
plt.title('Sum over camera image')
plt.xlabel('y_PositionSync')
plt.ylabel('Sum of %s:Image'%camname)
plt.semilogy()



plt.figure()
plt.plot(y_vals,im3d[:,thiscol,thisrow],'-o',label='each point')
plt.plot(y_vals,im3d_smooth[:,thiscol,thisrow],'-',linewidth=2,label='smoothed')
plt.plot(ygv,arr3d[:,thiscol,thisrow],'-k',label='interpolated')
plt.legend()
plt.xlabel('y_PositionSync')
plt.ylabel('Single pixel from %s:Image'%camname)
plt.semilogy()

In [ ]:
max1 = np.unravel_index(np.argmax(im3d),np.shape(im3d)); max1 = max1[0]
max2 = np.unravel_index(np.argmax(arr3d),np.shape(arr3d)); max2 = max2[0]+1

plt.figure(figsize=[20,5])
plt.subplot(141)
plt.pcolor(np.log10(im3d[max1,:,:]))
plt.title('raw image')
plt.subplot(142)
plt.pcolor(np.log10(arr3d[max2,:,:]))
plt.title('interpolated along y')
plt.subplot(143)
plt.plot(im3d[max1,:,:].sum(axis=0),label='raw')
plt.plot(arr3d[max2,:,:].sum(axis=0),label='interpolated')
plt.semilogy()
plt.legend()
plt.title('ProfileX')
plt.subplot(144)
plt.plot(im3d[-2,:,:].flatten(),'x',label='raw')
plt.plot(arr3d[-2,:,:].flatten(),'.',label='interpolated')
plt.legend()
plt.title('pixels in edge frame')
    
plt.figure(figsize=[11,10])
plt.subplot(221)
plt.pcolor(im3d[max1,:,:])
plt.title('raw image')
plt.subplot(222)
plt.pcolor(arr3d[max2,:,:])
plt.title('interpolated along y')

irow,icol = np.where(im3d[max1,:,:]==im3d[max1,:,:].max())
row1= irow[0]
col1 = icol[0]

irow,icol = np.where(arr3d[max2,:,:]==arr3d[max2,:,:].max())
row2= irow[0]
col2 = icol[0]

plt.subplot(223) 
plt.title('Slice in Cam06 X')
plt.semilogy(im3d[max1,row1,:]/im3d[max1,row1,:].max(),'.-')
plt.semilogy(arr3d[max2,row2,:]/arr3d[max2,row2,:].max(),'.-')
#plt.ylim([.01,1.01])

plt.subplot(224) 
plt.title('Slice in Cam06 Y')
plt.semilogy(im3d[max1,:,col1]/im3d[max1,:,col1].max(),'.-')
plt.semilogy(arr3d[max2,:,col2]/arr3d[max2,:,col2].max(),'.-')
#plt.ylim([.01,1.01])

## Save slit coordinates

In [ ]:
# ordering: y,xp,x
xp_x_n = np.vstack([gv1.flatten(),gv2.flatten()])
xp_x = np.matmul(M[1:,1:],xp_x_n) + center[1:]
gv_xp = xp_x[0,:].reshape(np.shape(gv1))
gv_x = xp_x[1,:].reshape(np.shape(gv2))

plt.figure(figsize=[10,9])
plt.plot(gv_x[:],gv_xp[:],'or-');
plt.plot(act_pts[2],act_pts[1],'.k-')

plt.xlabel('x1'); plt.ylabel('x2')



In [ ]:
# -- define y grid
npts_y = 32
y = ds_sc[:,'y_PositionSync']
ygrid = np.linspace(y.min(),y.max(),npts_y)




In [ ]:
XGV = np.repeat(np.reshape(gv_x,np.append(np.shape(gv_x),1)),npts_y,axis=2)
XPGV = np.repeat(np.reshape(gv_xp,np.append(np.shape(gv_xp),1)),npts_y,axis=2)
YGV = np.zeros(np.shape(XGV))
for i in range(len(GV[2])):
    for j in range(len(GV[1])):
        YGV[i,j,:] = ygrid

plt.figure(figsize=[20,4])
plt.plot(YGV.flatten(),'.-'); plt.title('y1')
plt.xlim([0,10000])
plt.figure(figsize=[20,4])
plt.plot(XGV.flatten(),'.-'); plt.title('x1')
plt.xlim([0,10000])
plt.figure(figsize=[20,4])
plt.plot(XPGV.flatten(),'.-'); plt.title('x2')
plt.xlim([0,10000])

In [ ]:
plt.figure(figsize=[20,10])
plt.plot(XGV.flatten(),XPGV.flatten(),'o-')

plt.xlabel('x1'); plt.ylabel('x2')

In [ ]:
coord_3d = np.stack([XGV,XPGV,YGV],axis=0)
np.shape(coord_3d)

savefilename = 'slit_coordinates_%s.npy'%(filename.split('.')[0])
np.save(savefilename, coord_3d)

#### Make a function that does all the above



In [ ]:
# -- define y grid
npts_y = 32
y = ds_sc[:,'y_PositionSync'].copy()
ygrid = np.linspace(y.min(),y.max(),npts_y)


## -- make 3D image
npix_y = ny
npix_x = nx



def make_image3d(idx,ygv,ny,nx,plotFlag=False,newfig=True):

    # -- build up 3D image (Cam06 image for every data-point in sweep)
    nframes = len(idx)
    im3d = np.zeros([nframes,ny,nx])
    for i in range(nframes):
        ima = make_image(idx[i],plotFlag=False,thr=None)
        im3d[i,:,:] = ima
        

    # -- y Position for each frame
    y_vals = ds_sc[:,'y_PositionSync'][idx]
    # -- put in increasing order  (needed for interpolation)
    idx_sort = np.argsort(y_vals)
    y_vals = y_vals[idx_sort]
    im3d = im3d[idx_sort,:,:]
    # -- deal with any repeating points (otherwise interpolate won't work)
    repeat_pts, = np.where(np.diff(y_vals)==0)
    counter = 1
    for i in reversed(range(len(repeat_pts))):
        y_vals[repeat_pts[i]] -= .0000001*counter
        counter += 1

    ## -- apply filter along 1st dimension (y_PositionSync) only
    im3d_smooth = im3d#ndimage.median_filter(im3d, size=(3,1,1), mode='nearest')

    # -- interpolate
    yf, ypf, xf = np.meshgrid(ygv,ypix,xpix,indexing='ij')
    newpoints = np.vstack([yf.flatten(),ypf.flatten(),xf.flatten()]).T
    arr3d = interpolate.interpn((y_vals,ypix,xpix), im3d_smooth, newpoints, method='linear', bounds_error=False, fill_value=0.)
    arr3d = np.reshape(arr3d,[npts_y,ny,nx])

    if plotFlag:
        if newfig:
            plt.figure()
        plt.plot(ygv,arr3d.sum(axis=1).sum(axis=1),'o-')
        plt.xlabel('y_PositionSync [mm]')
             
    return(arr3d)

iteration = 109
thisidx = idx_bin[iteration]
a3d = make_image3d(thisidx,ygrid,npix_y,npix_x,plotFlag=True)
plt.title('Interpolated signal dependence on y slit')

# plt.figure()
# plt.plot(ds_sc[idx_bin[iteration],'y_PositionSync'],ds_sc[idx_bin[iteration],'cam06_Integral'],'.-')
# plt.title('cam06:Integral')

## View projection and slice plots for datapoints in this 1 sweep. 
coordinates y1,y3,x3

In [ ]:
iteration = 111 + 2*17
thisidx = idx_bin[iteration]
a3d = make_image3d(thisidx,ygrid,npix_y,npix_x,plotFlag=True)
plt.title('Interpolated signal dependence on y slit')

In [ ]:
plt.figure(figsize=[12,10])
plt.subplot(221)
thisplot(a3d[:,:,306],labels=['y3','y1'])
plt.title('slice at x3=306 (pixel)')
plt.subplot(222)
thisplot(a3d.sum(axis=2),labels=['y3','y1'])
plt.title('projection onto y1,y2')
plt.subplot(223)
thisplot(a3d.sum(axis=0),labels=['x3','y3'])
plt.title('projection onto y3,x3')
plt.tight_layout()

# Cropping camera image

Do this to save disk space 

## Step 1: estimate crop area based on this one sweep near beam center

In [ ]:
# -- crop limits
ix3_0 = 150
ix3_1 = nx - 100 
iy3_0 = 0
iy3_1 = ny

def plotcrop(a3d_,croplist,bn):
    (ix3_0_,ix3_1_,iy3_0_,iy3_1_) = croplist
    x3proj = a3d_.sum(axis=0).sum(axis=0)
    y3proj = a3d_.sum(axis=0).sum(axis=1)
    improj = a3d_.sum(axis=0)
    
    plt.figure(figsize = [17,5])
    plt.subplot(131)
    plt.semilogy(x3proj,'.-')
    yl = plt.ylim()
    plt.plot([ix3_0_,ix3_0_],yl,'r')
    plt.plot([ix3_1_,ix3_1_],yl,'r')
    plt.plot(np.arange(ix3_0_,ix3_1_),x3proj[ix3_0_:ix3_1_])
    plt.ylim(yl)
    plt.title('Projection onto x3 axis, sweep %i'%bn)

    plt.subplot(132)
    plt.semilogy(y3proj,'.-')
    yl = plt.ylim()
    plt.plot([iy3_0_,iy3_0_],yl,'r')
    plt.plot([iy3_1_,iy3_1_],yl,'r')
    plt.plot(np.arange(iy3_0_,iy3_1_),y3proj[iy3_0_:iy3_1_])
    plt.ylim(yl)
    plt.title('Projection onto y3 axis, sweep %i'%bn)

    plt.subplot(133) 
    plt.pcolor(np.log10(improj))
    xl = plt.xlim()
    yl = plt.ylim()
    plt.plot(xl,[iy3_0_,iy3_0_],'r')
    plt.plot(xl,[iy3_1_,iy3_1_],'r')
    plt.plot([ix3_0_,ix3_0_],yl,'r')
    plt.plot([ix3_1_,ix3_1_],yl,'r')
    plt.xlim(xl)
    plt.ylim(yl)
    plt.title('Projection along y1 axis, sweep %i'%bn)
    
thisidx = idx_bin[iteration]
a3d = make_image3d(thisidx,ygrid,npix_y,npix_x,plotFlag=False)
plotcrop(a3d,[ix3_0,ix3_1,iy3_0,iy3_1],iteration)

In [ ]:
# -- try cropping on some random bins
for i in range(30):
    iteration_ = np.random.randint(len(steps))
    
    thisidx = idx_bin[iteration_]
    a3d = make_image3d(thisidx,ygrid,npix_y,npix_x,plotFlag=False)
    plotcrop(a3d,[ix3_0,ix3_1,iy3_0,iy3_1],iteration_)

In [ ]:
arrays_3d_shape = tuple([len(GV[2]),len(GV[1])]+list(np.shape(a3d)))
print(arrays_3d_shape)

print('Cropping image to x3 = [%i, %i], y3 = [%i, %i]'%(ix3_0,ix3_1,iy3_0,iy3_1))
crop_shape = list(arrays_3d_shape)
crop_shape[3] = iy3_1-iy3_0
crop_shape[4] = ix3_1-ix3_0
crop_shape = tuple(crop_shape)
print(crop_shape)

### Loop over all bins in x,x' positions
~ 1 minute per set

also apply cropping; make memmap that is cropped array size. 


In [ ]:
t0 = time()
savefilename = 'rawgrid_%s.mmp'%(filename.split('.')[0])
arrays_3d = np.memmap(savefilename,shape=crop_shape,dtype=np.uint32,mode='w+') # specifying integer type saves memory
for ii in range(len(GV[2])):
    for jj in range(len(GV[1])):
        try:
            thisidx = idx_bin[POINTS2D[ii,jj]-1]
            a3d_ = make_image3d(thisidx,ygrid,npix_y,npix_x)
            arrays_3d[ii,jj,:,:,:] = a3d_[:,iy3_0:iy3_1,ix3_0:ix3_1]
        except IndexError:
            print('No points in grid bin %i,%i'%(ii,jj))
            arrays_3d[ii,jj,:,:,:] = 0 
    
tf = time()
print('Elapsed time %.2f minutes'%((tf-t0)/60.))

# savefilename = 'rawgrid_%s.npy'%(filename.split('.')[0])
# np.save(savefilename, arrays_3d)

#print(sys.getsizeof(arrays_3d))
#np.savez_compressed(savefilename, a3d=arrays_3d, x1gv=GV[2], x2gv = GV1[1], M=M, centers=centers, y1gv = ygrid)
del arrays_3d

# View cropped and thresholded array. 
Remember, grid coordinates are in normalized frame

In [ ]:
a5d = np.memmap(savefilename,shape=crop_shape,dtype=np.uint32,mode='r')

## Slices across beam center

In [ ]:
a5d_max = a5d.max()
ix1,ix2,iy1,iy2,ix3 = np.where(a5d==a5d_max)
ix1 = ix1[0];  ix2 = ix2[0]; ix3 = ix3[0]; iy1 = iy1[0]; iy2 = iy2[0]
print('Center at index (%i,%i,%i,%i,%i), peak value %.2f'%(ix1,ix2,iy1,iy2,ix3,a5d_max))

In [ ]:
ima = a5d[ix1,ix2,iy1,:,:].copy()
labels=['x3','y3']
nx = np.shape(ima)[1]
ny = np.shape(ima)[0]

plt.figure(figsize=[12,5])
plt.subplot(121); plot_image(ima,log=False,nx_=nx,ny_=ny,labels=labels); 
plt.subplot(122); plot_image(ima,log=True,nx_=nx,ny_=ny,labels=labels); plt.colorbar()



In [ ]:
ima = a5d[ix1,ix2,:,iy2,:].copy()
labels=['x3','y1']
nx = np.shape(ima)[1]
ny = np.shape(ima)[0]

plt.figure(figsize=[12,5])
plt.subplot(121); plot_image(ima,log=False,nx_=nx,ny_=ny,labels=labels); 
plt.subplot(122); plot_image(ima,log=True,nx_=nx,ny_=ny,labels=labels); plt.colorbar()





In [ ]:
ima = a5d[ix1,:,iy1,iy2,:].copy()
labels=['x3','x2']
nx = np.shape(ima)[1]
ny = np.shape(ima)[0]

plt.figure(figsize=[12,5])
plt.subplot(121); plot_image(ima,log=False,nx_=nx,ny_=ny,labels=labels); 
plt.subplot(122); plot_image(ima,log=True,nx_=nx,ny_=ny,labels=labels); plt.colorbar()


In [ ]:
ima = a5d[:,ix2,iy1,iy2,:].copy()
labels=['x3','x1']
nx = np.shape(ima)[1]
ny = np.shape(ima)[0]

plt.figure(figsize=[12,5])
plt.subplot(121); plot_image(ima,log=False,nx_=nx,ny_=ny,labels=labels); 
plt.subplot(122); plot_image(ima,log=True,nx_=nx,ny_=ny,labels=labels); plt.colorbar()

In [ ]:
ima = a5d[:,:,iy1+2,iy2,ix3].copy()
labels=['x2','x1']
nx = np.shape(ima)[1]
ny = np.shape(ima)[0]

plt.figure(figsize=[12,5])
plt.subplot(121); plot_image(ima,log=False,nx_=nx,ny_=ny,labels=labels); 
plt.subplot(122); plot_image(ima,log=True,nx_=nx,ny_=ny,labels=labels); plt.colorbar()

In [ ]:
ima = a5d[ix1,ix2,:,:,ix3].copy()
labels=['y2','y1']
nx = np.shape(ima)[1]
ny = np.shape(ima)[0]

plt.figure(figsize=[12,5])
plt.subplot(121); plot_image(ima,log=False,nx_=nx,ny_=ny,labels=labels); 
plt.subplot(122); plot_image(ima,log=True,nx_=nx,ny_=ny,labels=labels); plt.colorbar()

In [ ]:
ima = a5d[:,ix2,:,iy2,ix3].copy()
labels=['y1','x1']
nx = np.shape(ima)[1]
ny = np.shape(ima)[0]

plt.figure(figsize=[12,5])
plt.subplot(121); plot_image(ima,log=False,nx_=nx,ny_=ny,labels=labels); 
plt.subplot(122); plot_image(ima,log=True,nx_=nx,ny_=ny,labels=labels); plt.colorbar()


## Projections

Note: this is without threshold applied, so backgrounds are elevated.

In [ ]:
# (ix1,ix2,iy1,iy2,ix3)
ima = a5d.sum(axis=1).sum(axis=2).sum(axis=2)
labels=['y1','x1']
nx = np.shape(ima)[1]
ny = np.shape(ima)[0]

plt.figure(figsize=[12,5])
plt.subplot(121); plot_image(ima,log=False,nx_=nx,ny_=ny,labels=labels); 
plt.subplot(122); plot_image(ima,log=True,nx_=nx,ny_=ny,labels=labels); plt.colorbar()


In [ ]:
# (ix1,ix2,iy1,iy2,ix3)
ima = a5d.sum(axis=2).sum(axis=2).sum(axis=2)
labels=['x2','x1']
nx = np.shape(ima)[1]
ny = np.shape(ima)[0]

plt.figure(figsize=[12,5])
plt.subplot(121); plot_image(ima,log=False,nx_=nx,ny_=ny,labels=labels); 
plt.subplot(122); plot_image(ima,log=True,nx_=nx,ny_=ny,labels=labels); plt.colorbar()



In [ ]:
# (ix1,ix2,iy1,iy2,ix3)
ima = a5d.sum(axis=0).sum(axis=0).sum(axis=2)
labels=['y2','y1']
nx = np.shape(ima)[1]
ny = np.shape(ima)[0]

plt.figure(figsize=[12,5])
plt.subplot(121); plot_image(ima,log=False,nx_=nx,ny_=ny,labels=labels); 
plt.subplot(122); plot_image(ima,log=True,nx_=nx,ny_=ny,labels=labels); plt.colorbar()



In [ ]:
# (ix1,ix2,iy1,iy2,ix3)
ima = a5d.sum(axis=0).sum(axis=1).sum(axis=2)
labels=['y3','x2']
nx = np.shape(ima)[1]
ny = np.shape(ima)[0]

plt.figure(figsize=[12,5])
plt.subplot(121); plot_image(ima,log=False,nx_=nx,ny_=ny,labels=labels); 
plt.subplot(122); plot_image(ima,log=True,nx_=nx,ny_=ny,labels=labels); plt.colorbar()


In [ ]:
# (ix1,ix2,iy1,iy2,ix3)
ima = a5d.sum(axis=1).sum(axis=1).sum(axis=1)
labels=['x3','x1']
nx = np.shape(ima)[1]
ny = np.shape(ima)[0]

plt.figure(figsize=[12,5])
plt.subplot(121); plot_image(ima,log=False,nx_=nx,ny_=ny,labels=labels); 
plt.subplot(122); plot_image(ima,log=True,nx_=nx,ny_=ny,labels=labels); plt.colorbar()



In [ ]:
del a5d